In [1]:
import numpy as np
import sys
import os
sys.path.insert(0, '/home/fbuonerba/codes/')
from mp_functions import upload_log_return, upload_factor_loadings, request_rates
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import statsmodels.api as sm
import cvxopt
from cvxopt import matrix, solvers
from cvxopt.blas import dot
from cvxopt.solvers import qp

solvers.options['show_progress'] = False

In [2]:
#load coin names - it removes those with some outliers in historical data
with open('/home/fbuonerba/codes/meta_data/new_coins.txt') as ff:
    coins=json.load(ff)
bad=['NPXS','MKR','VET','RHOC', 'ONT', 'ZIL', 'NANO', 'BAT','BCD','XTZ']
coins=np.array(coins)
where=[i for i in range(len(coins)) if coins[i] in bad]
coins=np.delete(coins, where)

prettybad=['BTC','BCN','DCR','BTG','BTM','XVG']
prettywhere=[i for i in range(len(coins)) if coins[i] in prettybad]
coins=np.delete(coins, prettywhere)
quotes=['BTC']
print(coins)

['ETH' 'XRP' 'BCH' 'EOS' 'XLM' 'LTC' 'ADA' 'XMR' 'IOTA' 'TRX' 'ETC' 'DASH'
 'NEO' 'XEM' 'BNB' 'ZEC' 'OMG' 'LSK' 'ZRX' 'QTUM' 'DOGE' 'BTS' 'DGB' 'ICX'
 'STEEM' 'AE' 'WAVES' 'SC' 'REP' 'PPT' 'GNT' 'STRAT']


In [ ]:
#loads daily log-returns from beginning to end.
#W keeps track of coin indices for which no historical
#data is available, and removes them from final returns array.
#it returns W and time-series of log-returns (without pathological ones).
#With the current 32 coins stored in the above array coins, W is always empty.
#It need not be if one adds further coins.

#INPUT: beginning, end
#OUTPUT: array W , time-series of log-returns

def get_returns(beg,end):
    t=beg
    matrix=[]
    while t<=end+1:
        ret_t=[]
        for base in coins:
            for quote in quotes:
                returns=upload_log_return(t, base, quote, 86400)
                if np.isnan(returns)==True:
                    returns=0
                ret_t.append(returns)
        matrix.append(ret_t)
        t+=86400
    R=np.array(matrix)
    norms=np.linalg.norm(R, axis=0)
    W=np.where(norms==0)
    R=np.delete(R, W, axis=1) 
    return(W, R)



In [ ]:
#loads factor loadings, computed using data from beginning to end
#the default time interval end-beg= 84 days.
#there are 3 indices: averaged, exact, naive. 
#Each correspond to a different technique to compute factor loadings.
#Averaged factors are those that performed best in tests; exact factors are as in Barra.
#W is the list of pathological coins as computed in get_returns
#Returns std of log-returns over time interval, and the factor loadings

#INPUT: beginning, end, array W from above, index. By default end-beg=84days
#OUTPUT: std of log-returns from beg to end, factor loadings

factors=[]
keys=['returns_variance', 'returns_strength', 'rates_high_low','turnover', 'log_marketcap']
naive_folder='/home/fbuonerba/factor_loadings/naive_factors_'
exact_folder='/home/fbuonerba/factor_loadings/exact_factors_'

def get_raw_factors(beg,end,W=[],index='averaged'):
    folder='/home/fbuonerba/factor_loadings/averaged_factors_'
    if index=='exact':
        folder=exact_folder
    elif index=='naive':
        folder=naive_folder
    factors=[]
    std=[]    
    R=get_returns(beg, end)[1].T
    for coin in coins:
        for quote in quotes:
            with open(folder+coin+'_'+quote+'_'+str(beg)+'_'+str(end)+'_86400.txt') as data:
                fac=json.load(data)
            ordr=[]
            for key in keys:
                if key=='returns_variance':
                    E=fac[key]**.5
                    std.append(E)
                elif key=='returns_strength':#log(p_T/p_t)
                    E=np.sum(R[np.where(coins==coin)])#summing over time
                else:    
                    E=fac[key]
                ordr.append(E)
            factors.append(ordr)
    factors=np.array(factors)
    std=np.array(std)
    factors=np.delete(factors, W, axis=0)
    std=np.delete(std, W, axis=0)
    std=std.reshape(-1,1)
    factors[np.where(np.isnan(factors)==True)]=0
    return(std, factors)





In [ ]:
#Processes factor loadings in two possible ways:
#preprocess 1: scales so that each factor has mean zero and std one in the coin direction
#preprocess 2: for each factor, coins are ranked by the value of the factor; then
#rankings are scaled to (-1,1) and these numbers replace the actual factor.

#INPUT: beginning, end, preprocess style, index of factor loadings
#OUTPUT: std of log-returns from beg to end, processed factor loadings

def get_processed_factors(beg,end,W,preprocess=1, index='averaged'):
    std, factors=get_raw_factors(beg,end,W, index)
    if preprocess==2:
        factors=np.argsort((np.argsort(factors, axis=0)), axis=0 )#order coins by factors 
        factors=2*factors/np.max(factors)-1 #scale so max=1, min=-1
    else:
        factors=(factors-np.mean(factors,axis=0))/np.std(factors,axis=0)
    return(std, factors)

In [ ]:
#Fits linear model for a given day and factor preprocessing style.
#it uses statsmodels standard package.

#INPUT: day, preprocess style, presence of weights in linear regression.
#OUTPUT:log-returns on day; processed factor loadings on day;
#std of log-returns over previous84 days; the whole WLS python object.

def fit_model(day, preprocess=1, weights='Yes'):
    W, returns=get_returns(day, day)
    yday=day-86400#use only past data to compute factor loadings
    returns=np.array(returns[0])#make returns array of correct dimension
    std, factors=get_processed_factors(yday-(1522540800-1515283200),yday, W, preprocess)
    factors=sm.add_constant(factors)
    wt=1/std**2
    if weights=='No':
        wt=np.ones_like(std)
    wls=sm.WLS(returns, factors, weights=wt).fit()
    return(returns, factors, std, wls)

In [ ]:
#compute covariance matrix using the linear model on a given day.
#for the previous 84 days, it computes the linear regression above.
#it remember two things: 
#1) time-series of factor returns of regression:
#since there are 5 factors, the total factor returns cover a 84x6 matrix, 6=5+1
#accounts for intercept term. 
#2) time-series of residuals of each regression. 
#finally cov = X (cov(factor_returns)) X^t + diagonal (variance(residuals))
#here X= factor loadings as computed on the last of the 84 days.

#INPUT: day, preprocess style
#OUTPUT: model covariance matrix

def cov(day, preprocess=1):
    day0=day-(1522540800-1515283200)
    BETA=[]
    RES=[]
    RET=[]
    days=int((1522540800-1515283200)/86400)
    for t in range(days):
        returns, factors, std, wls=fit_model(day0+86400*t, preprocess)
        BETA.append(wls.params)
        RES.append(returns-wls.predict() )
        RET.append(returns)
    BETA=np.array(BETA)
    RES=np.array(RES)
    RET=np.array(RET)
    cov=np.dot( np.dot(factors, np.cov(BETA.T)), factors.T) 
    varian=np.var(RES, axis=0)#tested everything has positive eigenvalues
    cov = cov + np.diag(varian)
    return(cov)

In [ ]:
#this is sample covariance matrix.
#It has been tested many times that this function and the previous one
#do operate on the very same set of log-returns (no shift in historical data).
#Moreover, all that is used to estimate cov(day) involves data up to day-1.

#INPUT: day
#OUTPUT:sample covarianec matrix

def raw_cov(day):  
    day0=day-(1522540800-1515283200)
    W, ret = get_returns(day0, day-86400)
    co=np.cov(ret.T)
    return(co)

In [ ]:
#statistics about covariance matrix: returns max,min eigenvalues, and their ratio.

#INPUT: covariance matrix
#OUTPUT: max, min eigenvalues, max/min
def covariance_stability(co):
    eigenvalues=np.linalg.svd(co)[1]
    mi=np.min(eigenvalues)
    ma=np.max(eigenvalues)
    condition=ma/mi
    return(ma, mi, condition)
    

In [ ]:
#builds optimal portfolio at given day using Markowitz theory.

#INPUT: day, alpha, linear constraints: Gx<h, Ax=b, preprocess style, risk aversion.
#OUTPUT: vector of weights for optimized portfolio.

#remarks: risk aversion is set to 10^3 so weights tend to be between -1 and 1.
#it solves quadratic problem -alpha.x + rho x^T.S.x (S=covariance matrix)

def portfolio(day, alpha, G,h,A,b, preprocess=2, rho=10**3):
    if preprocess==0:
        covv=raw_cov(day)
    else:
        covv=cov(day, preprocess)
    a=matrix(alpha)
    S=matrix(covv)
    G=matrix(G)
    h=matrix(h)
    #sol = solvers.qp(P,q,G,h,A,b)
    # minimize xPx + qx subject to
    #Gx < h
    #Ax = b (A,b can be omitted)
    if np.linalg.matrix_rank(A)==0:
        sol=np.array(qp(rho*S, -a, G, h)['x'])
    else:
        A=matrix(A)
        b=matrix(b)
        sol=np.array(qp(rho*S, -a, G, h, A, b)['x'])
    return(sol)



In [ ]:
#computes the total return of optimal portfolio held from day_init until day_end.

#INPUT: portfolio, day_init, day_end, lags in days (useless here)
#OUTPUT: portfolio, time-series of daily realized returns.
def portfolio_returns(portf, day_init, day_end, day_lag=0):
    days= int((day_end-day_init)/86400)
    lag=86400*day_lag
    ret=get_returns(day_init+lag, day_end+lag-1)[1]
    realized_returns=np.dot(ret, portf)
    return(portf, realized_returns)

In [ ]:
#computes statistics attached to the performance of the portfolio.

#INPUT: portfolio, time-series of its realized daily returns
#OUTPUT: mean, std, sharpe ratio of time-series of realized returns,
#gross and net values of the portfolio.
def portfolio_stats(portfolio, returns):
    mean=np.mean(returns)
    std=np.std(returns)
    sharpe=mean/std
    LBV=np.sum(portfolio.clip(min=0))
    SBV=-np.sum(portfolio.clip(max=0))
    gross=LBV+SBV
    net=LBV-SBV
    return(mean, std, sharpe, gross, net)
    



In [ ]:
#computes evolving portfolio in period of time between day_init and day_end.
#Everyday it creates optimal portfolio for that day, and returns an array of portfolios.
#It does so by running 'portfolio' function every day.

#INPUT: day_init, day_end, same other variables as portfolio.
#OUTPUT: time-series of daily optimized portfolios.

def evol_portfolio(day_init, day_end, alpha, G,h,A,b, preprocess=2, rho=10**3):
    portf=[]
    days=int((day_end-day_init)/86400)
    for t in range(days+1):#day_end is included.
        portf.append(portfolio(day_init+t*86400, alpha, G,h,A,b, preprocess, rho))
    portf=np.array(portf)[:,:,0].T
    return(portf)


In [ ]:
#computes realized returns of portfolio that is evolving every day.
#for every day, it computes dot(portfolio(day), returns(day)).
#there is a lag option, so that it could actually compute:
# dot (portfolio(day), returns(day+lag))

#INPUT: time-series of portfolios, day_init, day_end, number of days lag.
#OUTPUT: same time-series of portfolios, time-series of realized returns.

def evol_portfolio_returns(portf, day_init, day_end, day_lag=0):
    days= int((day_end-day_init)/86400)
    lag=86400*day_lag
    returns=get_returns(day_init+lag, day_end+lag)[1]
    einstein_returns=np.einsum('ij,ji->i', returns, portf)
    #realized_returns=np.dot(returns, portf).diagonal()---same as einsum.
    return(portf, einstein_returns)#returns latest portfolio and timeseries of returns
    
    

In [ ]:
#compute statistics attached to performance of evolving portfolio.

#INPUT: evolving portfolio, its realized returns time-series.
#OUTPUT: mean, std, sharpe ratio of realized returns, average turnover of portfolio,
#average gross, average net (all averages over time interval under consideration).

def evol_portfolio_stats(portfolio, returns):
    mean=np.mean(returns)
    std=np.std(returns)
    sharpe=mean/std
    diff= portfolio[:, 1:]-portfolio[:, :-1]
    avg_turnover=np.mean( np.sum(np.abs(diff), axis=0) )
    LBV=np.sum(portfolio.clip(min=0), axis=0)
    SBV=-np.sum(portfolio.clip(max=0), axis=0)
    gross=np.mean(LBV+SBV)
    net=np.mean(LBV-SBV)
    return(mean, std, sharpe, avg_turnover, gross, net)


In [ ]:
#Default parameters one can start with.
day=1522540800 + 85*86400#June 24
day0=day-(1522540800-1515283200)#April 1
num=32
G=-np.zeros((num,num))
h=np.zeros(num)
A=np.zeros(num).reshape(1,-1)
b=0.0
alpha=np.zeros(num)

In [ ]:
######################Chapter1######################

In [ ]:
FAC=[]
for t in range(90):
    FAC.append(get_processed_factors(day0+t*86400, day+t*86400,[],2)[1])
FAC=np.array(FAC)

In [ ]:
for c in range(32):
    for f in range(5):
        plt.plot(FAC[:,c,f])
    plt.show()

In [ ]:
R2=[]
BETA=[]
FAC=[]
RES=[]
RE=[]
manual_r2=[]
for t in range(90):
    rets,factors,std,wls=fit_model(day0+t*86400, 2)
    weights=std.reshape(1,-1)[0]**-1
    R2.append(wls.rsquared)
    BETA.append(wls.params)
    FAC.append(factors)
    RES.append(rets-wls.predict())
    RE.append(rets)
    ###statsmodels R2 computation
    RSS= np.sum((weights*(rets-wls.predict()))**2 )#weighted sum
    weighted_mean=np.average(rets, weights=weights**2)#weighted mean
    TSS=np.sum((weights*(rets-weighted_mean))**2)#weighted sum    
    manual_r2.append(1-RSS/TSS)
    ###
R2=np.array(R2)
manual_r2=np.array(manual_r2)
RE=np.array(RE)
RES=np.array(RES)
FAC=np.array(FAC)
BETA=np.array(BETA)

In [ ]:
print(np.mean(R2), np.max(np.abs(R2-manual_r2)))
plt.scatter(range(len(R2)), R2)
plt.scatter(range(len(R2)), manual_r2)
plt.show()
for c in range(32):
    print(coins[c])
    for j in range(6):
        FFF=FAC[:,c,j].T*BETA[:,j]
        print(np.corrcoef(RE.T[c], FFF))
        plt.plot(RE.T[5])
        plt.plot(FFF)
        plt.show()

In [ ]:
beta=np.array(BETA)
R2=np.array(R2)
FAC=np.array(FAC)

covv=cov(day0+86400*150)
covv2=cov2(day0+86400*150)
raw_covv=raw_cov(day0+86400*150)

corr=statsmodels.stats.moment_helpers.cov2corr(covv)
corr2=statsmodels.stats.moment_helpers.cov2corr(covv)
raw_corr=statsmodels.stats.moment_helpers.cov2corr(raw_covv)

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(20,20))
ZZZ=sns.heatmap(raw_corr[:15,:15], annot = True, linewidths=1.5, ax=ax,xticklabels=coins[:15], yticklabels=coins[:15])

In [ ]:
#factor loadings for each coin
for c in range(32):
    plt.plot(FAC[:,c])
    plt.show()

In [ ]:
#cumulative factor returns
cumbeta=np.cumsum(beta, axis=0)    
keyss=['intercept','returns_std', 'returns_strength', 'rates_high_low','turnover', 'log_marketcap',]
x=range(len(beta))
fig, ax = plt.subplots(figsize=(10,8))
for i in range(6):
    ax.plot(x,cumbeta[:,i], linewidth=1.5, label=keyss[i])
ax.legend(loc='lower right')    #plt.show()
#plt.savefig('factor_returns.png')

In [ ]:
R2=np.array(R2)
print(np.mean(R2), np.std(R2))
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(range(len(R2)), R2, color='black')
plt.axhline(y=np.mean(R2), color='red', linestyle='-', label='average R2 = 0.23')
ax.legend(loc='lower right')
#plt.savefig('r2_scatter.png')

In [ ]:
for t in range(10):
    dayt=day+t*7*86400
    co1=cov(dayt, 1)
    co2=cov(dayt, 2)
    co3=raw_cov(dayt)
    print('+++++++')
    print(covariance_stability(co1))
    print(covariance_stability(co2))
    print(covariance_stability(co3))

In [ ]:
############Chapter2###############

In [ ]:
for i in range(10):#ideally std/gross should be very small.
    alpha=np.random.randn(32)
    day30=day+86400*30
    port=portfolio(day, alpha, G,h,A,b, 2)
    port_sample=portfolio(day, alpha, G,h,A,b, 0)#sample
    ret=portfolio_returns(port, day, day30)[1]
    ret_sample=portfolio_returns(port_sample, day, day30)[1]
    st=portfolio_stats(port, ret)
    st_sample=portfolio_stats(port_sample, ret_sample)
    print(st[1]/st[3])
    print(st_sample[1]/st_sample[3])
    print('======')


In [ ]:
alpha=np.zeros(num)
for i in range(2,11):
    alpha[:i]=1
    portf=evol_portfolio(day, day+90*86400, alpha, G,h,A,b,'box')#up to Sept 22
    pr=evol_portfolio_returns(portf, day, day + 86400*90)
    portfraw=evol_portfolio(day, day+90*86400, alpha, G,h,A,b, 'raw')
    prraw=evol_portfolio_returns(portfraw, day, day + 86400*90)
    print('model', np.dot(portf, alpha))
   # rreturns=get_returns(day, day+90*86400)[1]
    print('sample', np.dot(portfraw, alpha))
    #print('alpha:', alpha)
    print('mean_realized_ret, std_realized_ret, sharpe, avg_turnover, avg_gross, avg_net')
    print('model_cov', evol_portfolio_stats(pr[0],pr[1]))
    print('sample_cov', evol_portfolio_stats(prraw[0],prraw[1]))
    #print('*****************************')
    plt.plot(np.cumsum(pr[1]))
    plt.plot(np.cumsum(prraw[1]))
    plt.show()
    

In [ ]:
for x in range(5):
    rho=10**-x
    alpha=np.zeros(32)
    alpha[:5] = 1
    port, pp = portfolio_returns(alpha, day, day+140*86400, None , 0.01)
    print(port.T)
    pp=np.array([x[0] for x in pp])
    cumpp=np.cumsum(pp)
    ssstd=[]
    mmmean=[]
    plt.plot(cumpp)
    plt.show()
    for t in range(1,140):
        ssstd.append(np.std(cumpp[:t]))
        mmmean.append(np.mean(cumpp[:t]))
    plt.plot(mmmean)
    plt.plot(ssstd)
    plt.show()


In [ ]:
alpha=np.zeros(32)
alpha[0]=1

#cc=np.array(coins).reshape(-1,1)
day=1522540800
day_end=day + 139*86400
#alpha/=np.sum(alpha)
port=portfolio(day_end, alpha)[0]
print(np.sum(np.abs(port)))
for h in range(32):
    print(coins[h+1], '&', round(port[h][0],2), '\\')

In [ ]:
def ro(x):
    return(round(x,4))

day=1522540800
day_end=day + 140*86400
alpha=np.zeros(32)
for i in range(1,10):
    alpha[:i]=1
    alpha/=np.sum(alpha)
    opt_m, opt_std, opt_sh=portfolio_stats(alpha, day, day_end)
    raw_m, raw_std, raw_sh=portfolio_stats(alpha, day, day_end,'raw')
    print(i,'&',ro(opt_m),'&',ro(raw_m),'&',ro(opt_std),'&',ro(raw_std),'&',ro(opt_sh),'&',ro(raw_sh))

In [ ]:
day=1522540800
r2=[]
beta=[]
inter=[]
retur=[]
errors=[]
for t in range(140):
    ret, _ , _ , wls=fit_model(day+86400*t)#, weights='No')
    #print(wls.params)
    #print(wls.summary())
    retur.append(ret)
    errors.append(ret-wls.predict())
    #r2.append(wls.rsquared)
    beta.append(wls.params)


In [ ]:
errors=np.array(errors)
et=errors.T
rt=(np.array(retur)).T
l=et.shape[1]
autoR=[]
autoE=[]
for i in range(32):
    autoE.append(np.corrcoef(et[i][1:], et[i][:-1])[0,1])
    autoR.append(np.corrcoef(rt[i][1:], rt[i][:-1])[0,1])
    print(np.mean(et[i][1:]- et[i][:-1]) / np.std(et[i][1:]- et[i][:-1]))
    print(np.mean(rt[i][1:]- rt[i][:-1]) / np.std(rt[i][1:]- rt[i][:-1]))
    print('+++++')
    #plt.scatter(range(l-1), np.random.normal(0,np.std(et[i]), l-1))
    #plt.scatter(range(l-1), et[i][1:]- et[i][:-1])
    plt.show()

print(np.mean(autoE))#/ np.std(autoE))
print(np.mean(autoR))#/ np.std(autoR))

In [ ]:
keyss=['intercept','returns_std', 'returns_strength', 'rates_high_low','turnover', 'log_marketcap',]
x=range(140)
beta=np.array(beta)
cumbeta=np.cumsum(beta.T, axis=1)
print(beta.T-cumbeta)
fig, ax = plt.subplots(figsize=(10,8))
#line1, = ax.plot(x, np.sin(x), '--', linewidth=2,
                 #label='Dashes set retroactively')
#line1.set_dashes(dashes)
for i in range(6):
    ax.plot(x,cumbeta[i], linewidth=1.5, label=keyss[i])
ax.legend(loc='lower right')    #plt.show()
plt.savefig('factor_returns.png')

In [ ]:

mr2=np.mean(r2)
print(mr2)
fig, ax = plt.subplots(figsize=(10,8))
#line1, = ax.plot(x, np.sin(x), '--', linewidth=2,
                 #label='Dashes set retroactively')
#line1.set_dashes(dashes)
ax.scatter(range(140), r2, color='black')
plt.axhline(y=mr2, color='red', linestyle='-', label='average R2 = 0.24')
ax.legend(loc='lower right')



plt.savefig('r2_scatter.png')

In [ ]:
fig, axs =plt.subplots(2,1)
clust_data = [coins, ]
collabel=("col 1", "col 2", "col 3")
axs[0].axis('tight')
axs[0].axis('off')
the_table = axs[0].table(cellText=clust_data,colLabels=collabel,loc='center')
print(coins)

In [ ]:
day=1522540800
alpha=np.zeros(32)
alpha[0]=1
RET=[]
RET_raw=[]
for i in range(1,10):
    alpha[:i]=1
    for t in range(140):
        RET.append(portfolio(day+t*86400, alpha)[1])
        RET_raw.append(portfolio(day+t*86400, alpha,'raw')[1])
        


In [ ]:
X=np.array(RET)-np.array(RET_raw)
us=0
them=0
for x in X:
    if x>0:
        us+=1
    else:
        them+=1
print(us, them)

In [ ]:
import statsmodels.api as sm
#stats for log_returns in terms of BTC
methods=['averaged','barra']
weights=[0,1]
T=list(range(20))
beg=[1522540800+t*604800 for t in T]
end=[1522540800+(t+1)*604800 for t in T]
beg1=[1515283200+t*604800 for t in T]
end1=[1522540800+t*604800 for t in T]
###statsmodels works if do one regression per day, i.e. y a vector###
ret=[]
errors=[]
predictions=[]
factor_loadings=[]
victory=0
victory1=0
victory2=0
rsquares=[]
print('r2_naive', 'r2_exact', 'r2_averaged')
for j in range(len(T)):
    W,R=get_returns(beg[j],end[j])
    print(W)
    sys.stdout.flush()
    std, factors=get_processed_factors(beg1[j],end1[j],W, 'naive')
    std, factors1=get_processed_factors(beg1[j],end1[j],W,'exact')
    std, factors2=get_processed_factors(beg1[j],end1[j],W,'averaged')
    print('**********')
    sys.stdout.flush()
    X=factors
    X1=factors1
    X2=factors2
    X = sm.add_constant(X)
    X1 = sm.add_constant(X1)
    X2 = sm.add_constant(X2)
    for i in range(7):
        Y=R[i].T
        ret.append(Y)
        #est_nowt=sm.WLS(Y,X).fit()
        sss=1/std**2
        est_wt=sm.WLS(Y,X, weights=sss).fit()
        est_wt1=sm.WLS(Y,X1, weights=sss).fit()
        est_wt2=sm.WLS(Y,X2, weights=sss).fit()
        beta2=est_wt2.params
        factor_loadings.append(beta2)
        Y_hat=est_wt2.predict()
        error=est_wt2.predict()-Y
        predictions.append(Y_hat)
        errors.append(error)
    #look for heteroskedasticity!
    #plt.scatter(est_nowt.predict(),error)
    #plt.scatter(est_wt.predict(), error)
    #plt.show()
    #print(est_wt.summary())
        r2=est_wt.rsquared
        r21=est_wt1.rsquared
        r22=est_wt2.rsquared
        rsquares.append(r22)
        #print(r2, r21, r22)
        argm=np.argmax(np.array([r21, r22]))
        if argm==0:
            victory+=1
        elif argm==1:
            victory1+=1
        else:
            victory2+=1
print(victory, victory1, victory2)
    
#print('times naive won:', victory, 'times barra won', victory1)
    
        
#    #check errors are independent of factors!
#     for k in range(X.shape[1]):
#         if k>0:
#             print(keys[k-1])
#         plt.scatter(range(len(X.T[k])),X.T[k])#, error)
#         plt.show()

####check that errors are more or less iid by plotting covariance matrix####

rsquares=np.array(rsquares)
print(rsquares)
print(np.min(rsquares), np.max(rsquares), np.mean(rsquares))

In [ ]:
# X2 in memory is factor loadings at final time - we can use that
import statsmodels
predictions=np.array(predictions)
errors=np.array(errors)
ret=np.array(ret)
factor_loadings=np.array(factor_loadings)
factor_cov=np.cov(factor_loadings.T)
#factor_cov.shape
est_covariance=np.dot( np.dot(X2, np.cov(factor_loadings.T)), X2.T) 
delta=np.var(errors, axis=0)
est_covariance+=np.sns.set()
fig, ax = plt.subplots(figsize=(20,20))
ZZZ=sns.heatmap(raw_corr[:15,:15], annot = True, linewidths=1.5, ax=ax,xticklabels=coins[:15], yticklabels=coins[:15])diag(delta)
raw_corr=np.corrcoef(ret.T)
raw_est_corr=np.corrcoef(predictions.T)
est_corr=statsmodels.stats.moment_helpers.cov2corr(est_covariance)
print('first plots Corr(raw_returns), second plots Corr(predicted_returns), \
third plots Factors.Corr(beta).Factors^T + Diag(err^2) ')
#print(raw_corr.shape, raw_est_corr.shape, est_corr.shape)
#heatmap of raw correlation

#ZZZ.figure.savefig("raw_corr.png")
#heatmap of raw correlation

sns.set()
fig, ax = plt.subplots(figsize=(20,20))
ZZZ=sns.heatmap(raw_est_corr[:15,:15], annot = True, linewidths=1.5, ax=ax,xticklabels=coins[:15], yticklabels=coins[:15])
#ZZZ.figure.savefig("raw_estimated_corr.png")
#heatmap of estimated correlation
sns.set()
fig, ax = plt.subplots(figsize=(20,20))
ZZZ=sns.heatmap(est_corr[:15,:15], annot = True, linewidths=1.5, ax=ax,xticklabels=coins[:15], yticklabels=coins[:15])
#ZZZ.figure.savefig("model_corr.png")

In [ ]:
corr_difference=est_corr-raw_corr
corr_difference
sns.set()
fig, ax = plt.subplots(figsize=(20,20))
ZZZ=sns.heatmap(corr_difference[:15,:15], vmin=-0.00001, vmax=0.00001, annot = True, linewidths=1.5, ax=ax,xticklabels=coins[:15], yticklabels=coins[:15])
ax.set_title('model_estimated_correlations - raw_correlations')
ZZZ.figure.savefig("correlation_difference.png")

In [ ]:
ret_tomorrow=ret[1:]
ret_today=ret[:-1]
err_tomorrow=errors[1:]
err_today=errors[:-1]
serial_ret_corr=np.corrcoef(ret_tomorrow.T , ret_today.T)[:32,32:].diagonal()
serial_error_corr=np.corrcoef(err_tomorrow.T , err_today.T)[:32,32:].diagonal()
ser_ret_m=np.mean(serial_ret_corr)
ser_ret_std=np.std(serial_ret_corr)
ser_err_m=np.mean(serial_error_corr)
ser_err_std=np.std(serial_error_corr)
print('mean and std for serial return correlations')
print(ser_ret_m, ser_ret_std)
print('mean and std for serial residual correlations')
print(ser_err_m, ser_err_std)
#plt.xlim(-1e-3,1e-3)
#plt.ylim(-1e-3,1e-3)
plt.ylabel('serial error corr')
plt.xlabel('serial return corr')
plt.xticks(rotation=90)
plt.scatter(serial_ret_corr,serial_error_corr)
# plt.savefig("serial_correlations.png")

In [ ]:
#same as above.
ser_corr_ret=[]
ser_corr_err=[]
for c in range(32):
    ser_corr_ret.append(np.corrcoef(ret.T[c][1:], ret.T[c][:-1])[0,1])
    ser_corr_err.append(np.corrcoef(errors.T[c][1:], errors.T[c][:-1])[0,1])
ser_corr_ret=np.array(ser_corr_ret)
plt.scatter(ser_corr_ret, ser_corr_err)
plt.savefig("serial.png")

In [ ]:
import scipy.odr
def f(B,x):
    return B[0]*x + B[1]
linear=scipy.odr.Model(f)
data=scipy.odr.Data(serial_ret_corr,serial_error_corr)#order is x,y for data.
odr = scipy.odr.ODR(data, linear, beta0=[0, 0])
out=odr.run()
out.pprint()

In [ ]:

for c in range(32):
    Y=ret.T[c].T
    Y_=Y[7:]
    matrix_c=np.append(Y[:7], np.zeros(133))
    for t in range(1,133):
        row=np.append(np.append(np.zeros(t), Y[t:t+7]), np.zeros(133-t))
        matrix_c=np.vstack((matrix_c, row.T))
#     print(Y_)
#     for j in range(133):
#         print(matrix_c[j])
    ols=sm.OLS(Y_, X).fit()
    print(ols.params)
    print(ols.summary())

In [ ]:
#autoregressive model for returns, weekly.
#R_t = linear function in (R_{t-1},...,R_{t-7})
for t in range(ret.shape[0]-7):
    Y=ret[t+7]
    X=ret[t:t+7].T
    ar7=sm.OLS(Y,X).fit()
    #print(ar7.rsquared)
import statsmodels.tsa.ar_model as AR
for t in range(32):
    lag=7
    Y=ret.T[t]
    ar=AR.AR(Y).fit(maxlag=lag)
    Y_hat=ar.predict()
    RES=Y[lag:]-Y_hat
    TSS=np.sum((Y-np.mean(Y))**2)
    RSS=np.sum((RES-np.mean(RES))**2)
    R2=1-RSS/TSS